# BERT SQuAD 1.0 Model

In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:20000]")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [ ]:
squad = squad.train_test_split(test_size=0.2)

In [ ]:
squad["train"][0]

{'id': '56d1179117492d1400aab90c',
 'title': 'New_York_City',
 'context': "New York City's public bus fleet is the largest in North America, and the Port Authority Bus Terminal, the main intercity bus terminal of the city, serves 7,000 buses and 200,000 commuters daily, making it the busiest bus station in the world.",
 'question': 'How many buses visit the Port Authority Bus Terminal each day?',
 'answers': {'text': ['7,000'], 'answer_start': [155]}}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# Define a function to extract the inputs and labels for fine-tuning bert from squad dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]  # Extract the questions from the squad dataset
    inputs = tokenizer(
        questions,
        examples["context"],   # Extract the context from the squad dataset
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")  # each tuple contains two integers that represent the start and end character positions of the corresponding token in the original input text.
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
# Calling the preprocess_function on squad dataset
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
# Collate the examples into a batch using data collator
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [ ]:
# Load the pre-trained bert model
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [ ]:
# Perform the training and set up the parameters
training_args = TrainingArguments(
    output_dir="qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/dennischan/qa_model into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.353700,1.161598
2,0.926600,1.102535
3,0.674100,1.131263


TrainOutput(global_step=3000, training_loss=1.1336800842285155, metrics={'train_runtime': 3800.1971, 'train_samples_per_second': 12.631, 'train_steps_per_second': 0.789, 'total_flos': 9406683242496000.0, 'train_loss': 1.1336800842285155, 'epoch': 3.0})

In [ ]:
question1 = "What are the symptoms of infecting covid?"
context1 = "Cough (68%), fever/chills (58%), and shortness of breath (37%) reported most often. Median number of days from symptom onset to laboratory confirmed COVID-19 diagnosis was 4 days (range 0-26 days)"

In [ ]:
from transformers import pipeline
question_answerer = pipeline("question-answering", model="qa_model")

answer1 = question_answerer(question=question1, context=context1)['answer']
question_answerer(question=question1, context=context1)

{'score': 0.33816319704055786,
 'start': 0,
 'end': 25,
 'answer': 'Cough (68%), fever/chills'}

In [ ]:
question2 = "What causes death from Covid-19?"
context2 = "Since the end of 2019 the Coronavirus Disease 2019 (COVID-19), caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), has spread globally affecting people worldwide. Patients with severe CO VID-19 require intensive care unit (ICU) admission for acute respiratory failure and over 10% need noninvasive and invasive mechanical ventilation."


In [ ]:
answer2 = question_answerer(question=question2, context=context2)['answer']
question_answerer(question=question2, context=context2)

{'score': 0.3249039053916931,
 'start': 73,
 'end': 120,
 'answer': 'severe acute respiratory syndrome coronavirus 2'}

In [ ]:
question3 = "Where is the origin of covid?"
context3 = "On December 31, 2019, the first case of what would later be known as SARS-CoV-2 was detected in the city of Wuhan, China. By January 10, 2022, more than 305 million people had been infected with COVID- 19, leading to more than 5.4 million deaths."


In [ ]:
answer3 = question_answerer(question=question3, context=context3)['answer']
question_answerer(question=question3, context=context3)

{'score': 0.48505425453186035,
 'start': 108,
 'end': 120,
 'answer': 'Wuhan, China'}

In [ ]:
question4 = "What is the origin of covid?"
context4 = "The coronavirus disease 2019 (COVID-19) is caused by a novel strain of SARS-CoV-2. More than 305 million people had been infected with the virus by January 10, 2022, leading to 5.4 million deaths."


In [ ]:
answer4 = question_answerer(question=question4, context=context4)['answer']
question_answerer(question=question4, context=context4)

{'score': 0.08649849891662598, 'start': 71, 'end': 81, 'answer': 'SARS-CoV-2'}

In [ ]:
question5 = "When was covid first discovered?"
context5 = "In December 2019, a new coronavirus disease named COVID-19 by the World Health Organization broke out in Wuhan, China. There is currently no evidence to support that these drugs may be effective in discouraging Covid-19."

In [ ]:
answer5 = question_answerer(question=question5, context=context5)['answer']
question_answerer(question=question5, context=context5)

{'score': 0.81943678855896, 'start': 3, 'end': 16, 'answer': 'December 2019'}

# Reference

Hugging Face. Question answering — transformers doc-
umentation. https : / / huggingface . co / docs /
transformers / tasks / question _ answering,
n.d. Accessed: Apr. 18, 2023.